# Generación de modelos

En este *notebook* se van a probar varios modelos de clasificación para ver qué tal se desempeña cada uno de ellos en el corpus que estamos analizando.

In [1]:

from src.classes import Corpus
from src.read_data import create_text_list

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

import pickle

In [2]:
# Creación del corpus
corpus = Corpus(create_text_list())

El corpus que manejamos consta de las siguientes obras:

In [3]:
df = corpus.corpus_content()
df

,Title,Author
0,Fortunata y Jacinta,Benito Pérez Galdós
1,Gerona,Benito Pérez Galdós
2,La corte de Carlos IV,Benito Pérez Galdós
3,La de Bringas,Benito Pérez Galdós
4,Marianela,Benito Pérez Galdós
5,Misericordia,Benito Pérez Galdós
6,Napoleón en Chamartín,Benito Pérez Galdós
7,Trafalgar,Benito Pérez Galdós
8,La prueba,Emilia Pardo Bazán
9,La tribuna,Emilia Pardo Bazán


### KNN

El primer modelos que vamos a probar consiste en el Knn. La clase corpus posee un método de prueba para cada uno de los algoritmos. Consiste en predecir obra a obra el autor de la misma cuando se conoce la autoría del resto del corpus. De este modo, se realiza una especie de validación para cada uno de ellos y de este modo se puede usar en un caso real.

En el caso de knn vamos a probar, no solo con el mecanismo explicado en el párrafo anterior, sino también calculando el número de obras correctas tomando distintos parámetros del valor k:

In [4]:
for i in range(1,10):
    df = corpus.train_knn(k=i, n=1000)
    print(f'k = {i}')
    print(df[df['author'] == df['prediction']]['prediction'].count())

k = 1
19
k = 2
18
k = 3
15
k = 4
16
k = 5
17
k = 6
14
k = 7
16
k = 8
15
k = 9
14


Eliminando lo valores de 1 y 2, vemos que el mejor estimador lo hace cuando k = 5, caso en el que acierta el 81% de las obras del corpus. Vamos a ver las obras en las que falla y vamos a guardar el modelo con estos parámetros:

In [4]:
knn_prediction = corpus.train_knn(k=5)
knn_prediction

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,Benito Pérez Galdós,469256
1,Gerona,Benito Pérez Galdós,Benito Pérez Galdós,71742
2,La corte de Carlos IV,Benito Pérez Galdós,Benito Pérez Galdós,81242
3,La de Bringas,Benito Pérez Galdós,Benito Pérez Galdós,83676
4,Marianela,Benito Pérez Galdós,Benito Pérez Galdós,60722
5,Misericordia,Benito Pérez Galdós,Emilia Pardo Bazán,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,Benito Pérez Galdós,90710
7,Trafalgar,Benito Pérez Galdós,Benito Pérez Galdós,59558
8,La prueba,Emilia Pardo Bazán,Benito Pérez Galdós,74508
9,La tribuna,Emilia Pardo Bazán,Emilia Pardo Bazán,77267


In [5]:
# Saving the prediction in csv
knn_prediction.to_csv('./data/model_results/knn_prediction.csv')

A continuación guardamos el modelo:

In [6]:
# save model
pickle.dump(KNeighborsClassifier(n_neighbors=5), open('./models/knn.pickle', "wb"))

### Decision tree
Con un Decision Tree Classifier se desenvuelve bastante mal, no obstante genera un comportamiento muy interesante. Veamos qué ocurre si tan solo se selecciona 1 en el parámetro de max_depth:

In [7]:
dt_prediction_1 = corpus.train_dt(n=1000, max_depth=1, random_state=42)
dt_prediction_1

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,Benito Pérez Galdós,469256
1,Gerona,Benito Pérez Galdós,Benito Pérez Galdós,71742
2,La corte de Carlos IV,Benito Pérez Galdós,Benito Pérez Galdós,81242
3,La de Bringas,Benito Pérez Galdós,Benito Pérez Galdós,83676
4,Marianela,Benito Pérez Galdós,Benito Pérez Galdós,60722
5,Misericordia,Benito Pérez Galdós,Benito Pérez Galdós,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,Juan Valera,90710
7,Trafalgar,Benito Pérez Galdós,Juan Valera,59558
8,La prueba,Emilia Pardo Bazán,Benito Pérez Galdós,74508
9,La tribuna,Emilia Pardo Bazán,Benito Pérez Galdós,77267


Y ahora con 2 (a partir de 2, el desempeño es similar):

In [8]:
dt_prediction_2 = corpus.train_dt(n=1000, max_depth=2, random_state=42)
dt_prediction_2

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,Emilia Pardo Bazán,469256
1,Gerona,Benito Pérez Galdós,Benito Pérez Galdós,71742
2,La corte de Carlos IV,Benito Pérez Galdós,Benito Pérez Galdós,81242
3,La de Bringas,Benito Pérez Galdós,Emilia Pardo Bazán,83676
4,Marianela,Benito Pérez Galdós,Emilia Pardo Bazán,60722
5,Misericordia,Benito Pérez Galdós,Emilia Pardo Bazán,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,Juan Valera,90710
7,Trafalgar,Benito Pérez Galdós,Juan Valera,59558
8,La prueba,Emilia Pardo Bazán,Benito Pérez Galdós,74508
9,La tribuna,Emilia Pardo Bazán,Benito Pérez Galdós,77267


In [9]:
# Guardamos el resultado:
dt_prediction_1.to_csv('./data/model_results/dt_prediction_1.csv')
dt_prediction_2.to_csv('./data/model_results/dt_prediction_2.csv')

In [10]:
# Guardamos el modelo:
pickle.dump(DecisionTreeClassifier(max_depth=1, random_state=42), open('./models/dt1.pickle', "wb"))
pickle.dump(DecisionTreeClassifier(max_depth=2, random_state=42), open('./models/dt2.pickle', "wb"))

### SVC

Estas son las predicciones que se obtienen con un modelo SVC. En este caso se ha usado solo el corpus de obras de Pérez Galdós con las de Pardo Bazán ya que son las más similares y en las que han tenido dificultades los algoritmos previos:

In [11]:
svc_prediction = corpus.train_svc(kernel = "poly", degree=3, coef0=1, C=100000);
svc_prediction

c:\Users\raimu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\raimu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\raimu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\raimu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataC

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,Benito Pérez Galdós,469256
1,Gerona,Benito Pérez Galdós,Benito Pérez Galdós,71742
2,La corte de Carlos IV,Benito Pérez Galdós,Benito Pérez Galdós,81242
3,La de Bringas,Benito Pérez Galdós,Benito Pérez Galdós,83676
4,Marianela,Benito Pérez Galdós,Benito Pérez Galdós,60722
5,Misericordia,Benito Pérez Galdós,Emilia Pardo Bazán,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,Benito Pérez Galdós,90710
7,Trafalgar,Benito Pérez Galdós,Benito Pérez Galdós,59558
8,La prueba,Emilia Pardo Bazán,Emilia Pardo Bazán,74508
9,La tribuna,Emilia Pardo Bazán,Emilia Pardo Bazán,77267


In [12]:
# Saving the prediction in csv
svc_prediction.to_csv('./data/model_results/svc_prediction.csv')

In [13]:
# save model
pickle.dump(SVC(kernel="poly", degree=3, coef0=1, C = 100000), open('./models/svc.pickle', "wb"))

### Clustering

Es interesante ver cómo se comporta el corpus ante un aprendizaje no supervisado. Vamos a decirle que busque 3 clusters en Kmeans. Este ha sido el resultado:

In [3]:
clustering_prediction = corpus.train_clustering(n=1000, n_clusters=3, n_init=10, random_state=42)
clustering_prediction

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,1,469256
1,Gerona,Benito Pérez Galdós,1,71742
2,La corte de Carlos IV,Benito Pérez Galdós,1,81242
3,La de Bringas,Benito Pérez Galdós,1,83676
4,Marianela,Benito Pérez Galdós,1,60722
5,Misericordia,Benito Pérez Galdós,1,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,1,90710
7,Trafalgar,Benito Pérez Galdós,1,59558
8,La prueba,Emilia Pardo Bazán,1,74508
9,La tribuna,Emilia Pardo Bazán,2,77267


In [4]:
# Saving the prediction in csv
clustering_prediction.to_csv('./data/model_results/clustering_prediction.csv')

In [5]:
# save model
pickle.dump(KMeans(n_clusters=3, n_init=10, random_state=42), open('./models/clustering.pickle', "wb"))

### Gradient Boosting Classifier
Este modelo, que he dejado para el final, lo clava:

In [17]:
gradient_boosting_prediction = corpus.train_gbc(n=1000, max_depth=2, n_estimators=10, learning_rate=1.0, random_state=42)
gradient_boosting_prediction

,title,author,prediction,tokens
0,Fortunata y Jacinta,Benito Pérez Galdós,Benito Pérez Galdós,469256
1,Gerona,Benito Pérez Galdós,Benito Pérez Galdós,71742
2,La corte de Carlos IV,Benito Pérez Galdós,Benito Pérez Galdós,81242
3,La de Bringas,Benito Pérez Galdós,Benito Pérez Galdós,83676
4,Marianela,Benito Pérez Galdós,Benito Pérez Galdós,60722
5,Misericordia,Benito Pérez Galdós,Benito Pérez Galdós,100143
6,Napoleón en Chamartín,Benito Pérez Galdós,Benito Pérez Galdós,90710
7,Trafalgar,Benito Pérez Galdós,Benito Pérez Galdós,59558
8,La prueba,Emilia Pardo Bazán,Emilia Pardo Bazán,74508
9,La tribuna,Emilia Pardo Bazán,Emilia Pardo Bazán,77267


In [18]:
# Saving the prediction in csv
gradient_boosting_prediction.to_csv('./data/model_results/gradient_boosting_prediction.csv')

In [19]:
# save model
pickle.dump(GradientBoostingClassifier(max_depth=2,
                                 n_estimators=10,
                                 learning_rate=1.0,
                                 random_state=42), open('./models/clustering.pickle', "wb"))